In [3]:
import pandas as pd
import time 
import os
import requests
from helpers import get_release_info

%load_ext autoreload
%autoreload 2

In [18]:
columns = ['Release_ID', 'Master_ID', 'Artists', 'Title', 'Format', \
           'Labels', 'Genres', 'Styles', 'Country', 'Released', \
           'Rating', 'Have_Want', 'Nb_for_sale', 'Lowest_price', \
           'Median_price', 'Highest_price', 'Nb_track', 'Tracklist', 'URL']


if os.path.exists('discogs_releases.p'):
    print('Discogs_releases dataframes found! Continue scrapping...\n')
    discogs = pd.read_pickle('discogs_releases.p')
    begin = discogs.loc[len(discogs)-1, 'Release_ID'] + 1
else:
    discogs = pd.DataFrame(columns=columns)
    begin = 1
discogs_tmp = pd.DataFrame(columns=columns)

for idx in range(begin, 10):    
    url = "https://api.discogs.com/releases/%s" % idx
    release = requests.get(url)
    print('%d : %s (status code: %d)' % (idx, url, release.status_code))

    if release.status_code == 429:
        print('Rate Limiting ! Wait 60 secondes...', end=' ')
        time.sleep(60)
        print('Done!')
        release = requests.get(url)
    
    if release.status_code != 404:
        discogs_tmp.loc[len(discogs_tmp)] = get_release_info(release.json())
    else:
        print('Release n°%d not found !' % idx)
    
    # To gain speed
    if len(discogs_tmp) > 1:
        discogs = pd.concat([discogs, discogs_tmp], ignore_index=True)
        discogs.to_pickle('discogs_releases.p')
        discogs_tmp = pd.DataFrame(columns=columns)
        
    

discogs = pd.concat([discogs, discogs_tmp], ignore_index=True)
discogs

3 : https://api.discogs.com/releases/3 (status code: 200)
4 : https://api.discogs.com/releases/4 (status code: 200)
5 : https://api.discogs.com/releases/5 (status code: 200)
6 : https://api.discogs.com/releases/6 (status code: 200)
Rate Limiting ! Wait 60 secondes... Done!
7 : https://api.discogs.com/releases/7 (status code: 200)
8 : https://api.discogs.com/releases/8 (status code: 200)
9 : https://api.discogs.com/releases/9 (status code: 200)


,Release_ID,Master_ID,Artists,Title,Format,Labels,Genres,Styles,Country,Released,Rating,Have_Want,Nb_for_sale,Lowest_price,Median_price,Highest_price,Nb_track,Tracklist,URL
0,1,NaN,[The Persuader],Stockholm,"[{'qty': '2', 'descriptions': ['12""', '33 ⅓ RP...","[{'name': 'Svek', 'entity_type': '1', 'catno':...",[Electronic],[Deep House],Sweden,1999,"{'count': 145, 'average': 4.49}","(498, 700)",16,7.976127,17.298351,24.772203,6,"[(, Östermalm, 4:45), (, Vasastaden, 6:11), (,...",https://www.discogs.com/The-Persuader-Stockhol...
1,2,713738,[Mr. James Barth & A.D.],Knockin' Boots Vol 2 Of 2,"[{'qty': '1', 'descriptions': ['12""', '33 ⅓ RP...","[{'name': 'Svek', 'entity_type': '1', 'catno':...",[Electronic],"[Broken Beat, Techno, Tech House]",Sweden,1998,"{'count': 86, 'average': 4.23}","(386, 304)",21,0.877307,4.982568,11.813508,4,"[(, A Sea Apart, 5:08), (, Dutchmaster, 4:21),...",https://www.discogs.com/Mr-James-Barth-AD-Knoc...
2,3,66526,[Josh Wink],Profound Sounds Vol. 1,"[{'descriptions': ['Compilation', 'Mixed'], 'n...","[{'name': 'Ruffhouse Records', 'entity_type': ...",[Electronic],"[Techno, Tech House]",US,1999,"{'count': 74, 'average': 4.3}","(198, 47)",16,0.984459,2.317162,8.886919,14,"[([Heiko Laux, Johannes Heil], Untitled 8, 7:0...",https://www.discogs.com/Josh-Wink-Profound-Sou...
3,4,104980,[Faze Action],Moving Cities,"[{'descriptions': ['Album'], 'name': 'CD', 'qt...","[{'name': 'F-111 Records', 'entity_type': '1',...",[Electronic],"[House, Deep House]",US,1999,"{'count': 29, 'average': 3.72}","(92, 30)",22,0.877307,3.462349,8.773070,12,"[(, Moving Cities, 6:24), (, To Love Is To Gro...",https://www.discogs.com/Faze-Action-Moving-Cit...
4,5,169944,[Datacide],Flowerhead,"[{'descriptions': ['Album', 'Limited Edition']...","[{'name': 'Rather Interesting', 'entity_type':...",[Electronic],"[Abstract, Ambient]",Germany,1995,"{'count': 58, 'average': 4.38}","(168, 143)",4,6.697000,24.423959,39.867241,5,"[(, Flashback Signal, 15:54), (, Flowerhead, 9...",https://www.discogs.com/Datacide-Flowerhead/re...
5,6,713737,[Mr. James Barth & A.D.],Knockin' Boots (Vol 1 Of 2),"[{'descriptions': ['12""', '33 ⅓ RPM'], 'name':...","[{'name': 'Svek', 'entity_type': '1', 'catno':...",[Electronic],"[Techno, Tech House]",Sweden,1998,"{'count': 142, 'average': 4.42}","(521, 638)",21,9.971833,21.684886,29.908802,3,"[(, It's All James, 5:29), (, Above The Skylin...",https://www.discogs.com/Mr-James-Barth-AD-Knoc...
6,7,267770,[Moonchildren],Moonchildren EP,"[{'descriptions': ['12""', 'EP'], 'text': 'Clea...","[{'name': 'Deep4Life', 'entity_type': '1', 'ca...",[Electronic],[Deep House],US,2000,"{'count': 66, 'average': 4.5}","(190, 300)",8,11.103626,18.805176,22.830073,3,"[(, Ran Away, 6:45), (, Silent Crying, 6:28), ...",https://www.discogs.com/Moonchildren-Moonchild...
7,8,48282,[Sweet Abraham],Spreading Outward EP,"[{'descriptions': ['12""', 'EP'], 'name': 'Viny...","[{'name': 'Deep4Life', 'entity_type': '1', 'ca...",[Electronic],[Deep House],US,2000,"{'count': 49, 'average': 4.22}","(95, 306)",3,9.971833,21.229490,28.910949,3,"[(, Diaspora, 7:15), (, Wallflower, 7:25), (, ...",https://www.discogs.com/Sweet-Abraham-Spreadin...
8,9,595959,[Blue Six],Pure,"[{'descriptions': ['12""', '33 ⅓ RPM'], 'name':...","[{'name': 'Naked Music Recordings', 'entity_ty...",[Electronic],"[House, Deep House]",US,2000,"{'count': 271, 'average': 4.51}","(1153, 457)",45,2.993559,6.656818,17.947960,4,"[(, Pure (Mig's Petalpusher Vocal), 7:02), (, ...",https://www.discogs.com/Blue-Six-Pure/release/9


In [ ]:
# # For test purpose...
# url = "https://api.discogs.com/masters/%s" % 478247
# url = "https://api.discogs.com/releases/%s" % 3922784
# release = requests.get(url)
# print('%d : %s (status code: %d)' % (idx, url, release.status_code))
